In [2]:
import warnings
warnings.filterwarnings('ignore')

In [1]:
# # EXAMPLE CODE

# # IMPORTS:
# from wav_conversion import convert_mp3_to_wav
# from textgrid_creation import create_textgrids
# from custom_mfa_align import align
# from formant_extraction import extract

# # ARGUMENTS:
# mp3_folder = "/home/henrynomeland/Documents/senior-thesis/speech-class/mp3-input"
# in_folder = "/home/henrynomeland/Documents/senior-thesis/speech-class/montreal-input"
# out_folder = "/home/henrynomeland/Documents/senior-thesis/speech-class/montreal-output"
# home_folder = "/home/henrynomeland/Documents/senior-thesis/speech-class" 
# label = "Please call Stella. Ask her to bring these things with her from the store: \
# Six spoons of fresh snow peas, five thick slabs of blue cheese, and maybe a snack for \
# her brother Bob. We also need a small plastic snake and a big toy frog for the kids. She \
# can scoop these things into three red bags, and we will go meet her Wednesday at the train station."
# acoustic_model = "english_us_arpa"
# dictionary_model = "english_us_arpa"
# filename="testing.csv"

# # this converts mp3 files to wav files
# # convert_mp3_to_wav(mp3_folder, in_folder, out_folder)
# # this creates the textgrids
# create_textgrids(label, in_folder)
# # this aligns the textgrids
# align(in_folder, out_folder, acoustic_model, dictionary_model)
# # this calculates formant measurements
# extract(out_folder, home_folder, filename)

In [6]:
from speech_modeling import h_input, h_model
import pandas as pd
df = pd.read_csv('gmu-formants.csv')
input_data = h_input(df)
input_data.process('region')
input_data.select_features(selected_features=['F1', 'F2', 'F3', 'duration'])
input_data.normalize(method='z')
input_data.select_places(places=['uk', 'texas', 'california', 'newyork'])

model_data = input_data.input_df
model_features = ['age', 'gender'] + list(input_data.input_df.columns[4:])

m = h_model(data=model_data, features=model_features, y_feature="location", y_main="uk")
m.fit(NUM_SAMPLES=1)

Fitting Model of type rforest
Modeling Sample 1 of 1

Accuracy: 0.942

Variable Importance Measurements:
        Feature  Importance
154    du-store    0.001036
111     F2-with    0.000740
129     du-frog    0.000666
3       F1-also    0.000666
77       F2-her    0.000592
65       F2-bob    0.000592
140  du-plastic    0.000518
32       F1-red    0.000444
37     F1-small    0.000370
13      F1-call    0.000370
